# Speech Recognition using Whisper




# **Introduction**

Whisper is an automatic speech recognition (ASR) system trained on 680,000 hours of multilingual and multitask supervised data collected from the web. We show that the use of such a large and diverse dataset leads to improved robustness to accents, background noise and technical language. Moreover, it enables transcription in multiple languages, as well as translation from those languages into English. We are open-sourcing models and inference code to serve as a foundation for building useful applications and for further research on robust speech processing.

In this example we will transcribe an audio clip locally, containerize the script and then 
Run the container on bacalhau

The advantages of using bacalhau over managed Automatic Speech Recognition services is that you can run your own containers which can scale to do batch process petabytes of Videos, Audio for automatic speech recognition, Using our sharding feature you can do distributed inference very easily and if you have the data stored on IPFS you don't need to move the data you can do compute where the data is located, and the cost of compute is much cheaper than managed services 



# Running whisper locally

Installing dependecies like Whisper, torch, pandas

In [ ]:
%%bash
pip install git+https://github.com/openai/whisper.git
pip install torch==1.10.1
pip install pandas
sudo apt update && sudo apt install ffmpeg

## Running the script

Before we create and run the script we need a sample audio file to test the code

for that we Download a sample audio clip



In [ ]:
%%bash
wget https://github.com/js-ts/hello/raw/main/hello.mp3

--2022-10-10 03:16:18--  https://github.com/js-ts/hello/raw/main/hello.mp3
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/js-ts/hello/main/hello.mp3 [following]
--2022-10-10 03:16:19--  https://raw.githubusercontent.com/js-ts/hello/main/hello.mp3
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10063 (9.8K) [audio/mpeg]
Saving to: ‘hello.mp3’

     0K .........                                             100% 52.3M=0s

2022-10-10 03:16:19 (52.3 MB/s) - ‘hello.mp3’ saved [10063/10063]




We will create a script that accepts parameters (input file path, output file path, temperature etc.) and set the default parameters. Also:
* If input file is in mp4 format, than the script converts it to wav format. 
* Save the transcript in various formats, 
* We load the large model
* Then pass it the required parameters.
This model is not only limited to english and transcription, it supports other languages and also does translation, to the following languages:

![](https://i.imgur.com/ALFe4qJ.png)

The graph above is sorted in [Word Error Rate (WER)](https://huggingface.co/spaces/evaluate-metric/wer) order.

Next, lets create a openai-whisper script:

In [ ]:
%%writefile openai-whisper.py
import argparse
import os
import sys
import warnings
import whisper
from pathlib import Path
import subprocess
import torch
import shutil
import numpy as np
parser = argparse.ArgumentParser(description="OpenAI Whisper Automatic Speech Recognition")
parser.add_argument("-l",dest="audiolanguage", type=str,help="Language spoken in the audio, use Auto detection to let Whisper detect the language. Select from the following languages['Auto detection', 'Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Assamese', 'Azerbaijani', 'Bashkir', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Breton', 'Bulgarian', 'Burmese', 'Castilian', 'Catalan', 'Chinese', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Estonian', 'Faroese', 'Finnish', 'Flemish', 'French', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hungarian', 'Icelandic', 'Indonesian', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Lao', 'Latin', 'Latvian', 'Letzeburgesch', 'Lingala', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Moldavian', 'Moldovan', 'Mongolian', 'Myanmar', 'Nepali', 'Norwegian', 'Nynorsk', 'Occitan', 'Panjabi', 'Pashto', 'Persian', 'Polish', 'Portuguese', 'Punjabi', 'Pushto', 'Romanian', 'Russian', 'Sanskrit', 'Serbian', 'Shona', 'Sindhi', 'Sinhala', 'Sinhalese', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog', 'Tajik', 'Tamil', 'Tatar', 'Telugu', 'Thai', 'Tibetan', 'Turkish', 'Turkmen', 'Ukrainian', 'Urdu', 'Uzbek', 'Valencian', 'Vietnamese', 'Welsh', 'Yiddish', 'Yoruba'] ",default="English")
parser.add_argument("-p",dest="inputpath", type=str,help="Path of the input file",default="/hello.mp3")
parser.add_argument("-v",dest="typeverbose", type=str,help="Whether to print out the progress and debug messages. ['Live transcription', 'Progress bar', 'None']",default="Live transcription")
parser.add_argument("-g",dest="outputtype", type=str,help="Type of file to generate to record the transcription. ['All', '.txt', '.vtt', '.srt']",default="All")
parser.add_argument("-s",dest="speechtask", type=str,help="Whether to perform X->X speech recognition (`transcribe`) or X->English translation (`translate`). ['transcribe', 'translate']",default="transcribe")
parser.add_argument("-n",dest="numSteps", type=int,help="Number of Steps",default=50)
parser.add_argument("-t",dest="decodingtemperature", type=int,help="Temperature to increase when falling back when the decoding fails to meet either of the thresholds below.",default=0.15 )
parser.add_argument("-b",dest="beamsize", type=int,help="Number of Images",default=5)
parser.add_argument("-o",dest="output", type=str,help="Output Folder where to store the ouputs",default="")

args=parser.parse_args()
device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

Model = 'large'
whisper_model =whisper.load_model(Model)
video_path_local = os.getcwd()+args.inputpath
file_name=os.path.basename(video_path_local)
output_file_path=args.output

if os.path.splitext(video_path_local)[1] == ".mp4":
    video_path_local_wav =os.path.splitext(file_name)[0]+".wav"
    result  = subprocess.run(["ffmpeg", "-i", str(video_path_local), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local_wav)])

# add language parameters
# Language spoken in the audio, use Auto detection to let Whisper detect the language.
#  ['Auto detection', 'Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Assamese', 'Azerbaijani', 'Bashkir', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Breton', 'Bulgarian', 'Burmese', 'Castilian', 'Catalan', 'Chinese', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Estonian', 'Faroese', 'Finnish', 'Flemish', 'French', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hungarian', 'Icelandic', 'Indonesian', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Lao', 'Latin', 'Latvian', 'Letzeburgesch', 'Lingala', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Moldavian', 'Moldovan', 'Mongolian', 'Myanmar', 'Nepali', 'Norwegian', 'Nynorsk', 'Occitan', 'Panjabi', 'Pashto', 'Persian', 'Polish', 'Portuguese', 'Punjabi', 'Pushto', 'Romanian', 'Russian', 'Sanskrit', 'Serbian', 'Shona', 'Sindhi', 'Sinhala', 'Sinhalese', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog', 'Tajik', 'Tamil', 'Tatar', 'Telugu', 'Thai', 'Tibetan', 'Turkish', 'Turkmen', 'Ukrainian', 'Urdu', 'Uzbek', 'Valencian', 'Vietnamese', 'Welsh', 'Yiddish', 'Yoruba']
language = args.audiolanguage
# Whether to print out the progress and debug messages.
# ['Live transcription', 'Progress bar', 'None']
verbose = args.typeverbose
#  Type of file to generate to record the transcription.
# ['All', '.txt', '.vtt', '.srt']
output_type = args.outputtype
# Whether to perform X->X speech recognition (`transcribe`) or X->English translation (`translate`).
# ['transcribe', 'translate']
task = args.speechtask
# Temperature to use for sampling.
temperature = args.decodingtemperature
#  Temperature to increase when falling back when the decoding fails to meet either of the thresholds below.
temperature_increment_on_fallback = 0.2
#  Number of candidates when sampling with non-zero temperature.
best_of = 5
#  Number of beams in beam search, only applicable when temperature is zero.
beam_size = args.beamsize
# Optional patience value to use in beam decoding, as in [*Beam Decoding with Controlled Patience*](https://arxiv.org/abs/2204.05424), the default (1.0) is equivalent to conventional beam search.
patience = 1.0
# Optional token length penalty coefficient (alpha) as in [*Google's Neural Machine Translation System*](https://arxiv.org/abs/1609.08144), set to negative value to uses simple length normalization.
length_penalty = -0.05
# Comma-separated list of token ids to suppress during sampling; '-1' will suppress most special characters except common punctuations.
suppress_tokens = "-1"
# Optional text to provide as a prompt for the first window.
initial_prompt = ""
# if True, provide the previous output of the model as a prompt for the next window; disabling may make the text inconsistent across windows, but the model becomes less prone to getting stuck in a failure loop.
condition_on_previous_text = True
#  whether to perform inference in fp16.
fp16 = True
#  If the gzip compression ratio is higher than this value, treat the decoding as failed.
compression_ratio_threshold = 2.4
# If the average log probability is lower than this value, treat the decoding as failed.
logprob_threshold = -1.0
# If the probability of the <|nospeech|> token is higher than this value AND the decoding has failed due to `logprob_threshold`, consider the segment as silence.
no_speech_threshold = 0.6

verbose_lut = {
    'Live transcription': True,
    'Progress bar': False,
    'None': None
}

args = dict(
    language = (None if language == "Auto detection" else language),
    verbose = verbose_lut[verbose],
    task = task,
    temperature = temperature,
    temperature_increment_on_fallback = temperature_increment_on_fallback,
    best_of = best_of,
    beam_size = beam_size,
    patience=patience,
    length_penalty=(length_penalty if length_penalty>=0.0 else None),
    suppress_tokens=suppress_tokens,
    initial_prompt=(None if not initial_prompt else initial_prompt),
    condition_on_previous_text=condition_on_previous_text,
    fp16=fp16,
    compression_ratio_threshold=compression_ratio_threshold,
    logprob_threshold=logprob_threshold,
    no_speech_threshold=no_speech_threshold
)

temperature = args.pop("temperature")
temperature_increment_on_fallback = args.pop("temperature_increment_on_fallback")
if temperature_increment_on_fallback is not None:
    temperature = tuple(np.arange(temperature, 1.0 + 1e-6, temperature_increment_on_fallback))
else:
    temperature = [temperature]

if Model.endswith(".en") and args["language"] not in {"en", "English"}:
    warnings.warn(f"{Model} is an English-only model but receipted '{args['language']}'; using English instead.")
    args["language"] = "en"

video_transcription = whisper.transcribe(
    whisper_model,
    str(video_path_local),
    temperature=temperature,
    **args,
)

# Save output
writing_lut = {
    '.txt': whisper.utils.write_txt,
    '.vtt': whisper.utils.write_vtt,
    '.srt': whisper.utils.write_txt,
}

if output_type == "All":
    for suffix, write_suffix in writing_lut.items():
        transcript_local_path =os.getcwd()+output_file_path+'/'+os.path.splitext(file_name)[0] +suffix
        with open(transcript_local_path, "w", encoding="utf-8") as f:
            write_suffix(video_transcription["segments"], file=f)
        try:
            transcript_drive_path =file_name
        except:
            print(f"**Transcript file created: {transcript_local_path}**")
else:
    transcript_local_path =output_file_path+'/'+os.path.splitext(file_name)[0] +output_type

    with open(transcript_local_path, "w", encoding="utf-8") as f:
        writing_lut[output_type](video_transcription["segments"], file=f)

Overwriting openai-whisper.py



Let's run the script with the default parameters:



In [ ]:
%%bash
python openai-whisper.py

[00:00.000 --> 00:00.500]  Hello!


Using device: cuda:0

  0%|                                              | 0.00/2.87G [00:00<?, ?iB/s]
  0%|                                     | 4.75M/2.87G [00:00<01:03, 48.7MiB/s]
  0%|                                     | 9.40M/2.87G [00:00<01:04, 48.0MiB/s]
  0%|▏                                    | 14.0M/2.87G [00:00<01:07, 45.8MiB/s]
  1%|▏                                    | 18.8M/2.87G [00:00<01:04, 47.4MiB/s]
  1%|▎                                    | 23.3M/2.87G [00:00<01:04, 47.3MiB/s]
  1%|▎                                    | 27.8M/2.87G [00:00<01:06, 45.8MiB/s]
  1%|▍                                    | 32.5M/2.87G [00:00<01:05, 46.6MiB/s]
  1%|▍                                    | 37.1M/2.87G [00:00<01:05, 46.9MiB/s]
  1%|▌                                    | 41.9M/2.87G [00:00<01:03, 47.7MiB/s]
  2%|▌                                    | 47.3M/2.87G [00:01<01:00, 50.2MiB/s]
  2%|▋                                    | 52.1M/2.87G [00:01<01:04, 47.2MiB/s]
  2%|▋

```
usage: openai-whisper.py [-h] [-l AUDIOLANGUAGE] [-p INPUTPATH]
                         [-v TYPEVERBOSE] [-g OUTPUTTYPE] [-s SPEECHTASK]
                         [-n NUMSTEPS] [-t DECODINGTEMPERATURE] [-b BEAMSIZE]
                         [-o OUTPUT]

OpenAI Whisper Automatic Speech Recognition

optional arguments:
  -h, --help            show this help message and exit
  -l AUDIOLANGUAGE      Language spoken in the audio, use Auto detection to
                        let Whisper detect the language. Select from the
                        following languages['Auto detection', 'Afrikaans',
                        'Albanian', 'Amharic', 'Arabic', 'Armenian',
                        'Assamese', 'Azerbaijani', 'Bashkir', 'Basque',
                        'Belarusian', 'Bengali', 'Bosnian', 'Breton',
                        'Bulgarian', 'Burmese', 'Castilian', 'Catalan',
                        'Chinese', 'Croatian', 'Czech', 'Danish', 'Dutch',
                        'English', 'Estonian', 'Faroese', 'Finnish',
                        'Flemish', 'French', 'Galician', 'Georgian', 'German',
                        'Greek', 'Gujarati', 'Haitian', 'Haitian Creole',
                        'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hungarian',
                        'Icelandic', 'Indonesian', 'Italian', 'Japanese',
                        'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean',
                        'Lao', 'Latin', 'Latvian', 'Letzeburgesch', 'Lingala',
                        'Lithuanian', 'Luxembourgish', 'Macedonian',
                        'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori',
                        'Marathi', 'Moldavian', 'Moldovan', 'Mongolian',
                        'Myanmar', 'Nepali', 'Norwegian', 'Nynorsk',
                        'Occitan', 'Panjabi', 'Pashto', 'Persian', 'Polish',
                        'Portuguese', 'Punjabi', 'Pushto', 'Romanian',
                        'Russian', 'Sanskrit', 'Serbian', 'Shona', 'Sindhi',
                        'Sinhala', 'Sinhalese', 'Slovak', 'Slovenian',
                        'Somali', 'Spanish', 'Sundanese', 'Swahili',
                        'Swedish', 'Tagalog', 'Tajik', 'Tamil', 'Tatar',
                        'Telugu', 'Thai', 'Tibetan', 'Turkish', 'Turkmen',
                        'Ukrainian', 'Urdu', 'Uzbek', 'Valencian',
                        'Vietnamese', 'Welsh', 'Yiddish', 'Yoruba']
  -p INPUTPATH          Path of the input file
  -v TYPEVERBOSE        Whether to print out the progress and debug messages.
                        ['Live transcription', 'Progress bar', 'None']
  -g OUTPUTTYPE         Type of file to generate to record the transcription.
                        ['All', '.txt', '.vtt', '.srt']
  -s SPEECHTASK         Whether to perform X->X speech recognition
                        (`transcribe`) or X->English translation
                        (`translate`). ['transcribe', 'translate']
  -n NUMSTEPS           Number of Steps
  -t DECODINGTEMPERATURE
                        Temperature to increase when falling back when the
                        decoding fails to meet either of the thresholds below.
  -b BEAMSIZE           Number of Images
  -o OUTPUT             Output Folder where to store the ouputs
```

Viewing the outputs


In [ ]:
%%bash
cat hello.srt

Hello!


# Building and Running on docker



In this step we will create a `Dockerfile` to create your Docker deployment. 

Next, add your desired configuration to the `Dockerfile`. These commands specify how the image will be built, and what extra requirements will be included.


```
FROM  pytorch/pytorch:1.12.1-cuda11.3-cudnn8-runtime

WORKDIR /

RUN apt-get -y update

RUN apt-get -y install git

RUN python3 -m pip install --upgrade pip

RUN python -m pip install regex tqdm Pillow

RUN pip install git+https://github.com/openai/whisper.git

ADD hello.mp3 hello.mp3

ADD openai-whisper.py openai-whisper.py

RUN python openai-whisper.py
```


We choose `pytorch/pytorch:1.12.1-cuda11.3-cudnn8-runtime` as our base image

And then install all the dependencies, after that we will add the test audio file and our openai-whisper script to the container, we will also run a test command to check whether our script works inside the container and if the container builds successfully



# Running whisper on Bacalhau

We will transcribe the moon landing video, which can be found here

https://www.nasa.gov/multimedia/hd/apollo11_hdpage.html

Since the downloaded video is in mov format we convert the video to mp4 format, and then upload it to IPFS.

To upload the video we will be using [https://nft.storage/docs/how-to/nftup/](https://nft.storage/docs/how-to/nftup/)

![](https://i.imgur.com/xwZT3Pi.png)

After the dataset has been uploaded, copy the CID:

`bafybeielf6z4cd2nuey5arckect5bjmelhouvn5rhbjlvpvhp7erkrc4nu` 

Let's run the container on Bacalhau. We use the `--gpu` flag to denote the no of GPU we are going to use:


```
bacalhau docker run \
jsacex/whisper \
--gpu 1 \
-i bafybeielf6z4cd2nuey5arckect5bjmelhouvn5rhbjlvpvhp7erkrc4nu \
-- python openai-whisper.py -p inputs/Apollo_11_moonwalk_montage_720p.mp4 -o outputs
```

In the command above we use:

* `-i bafybeielf6z4cd2nuey5arckect5bjmelhouvn5r` flag to mount the CID which contains our file to the container at the path `/inputs`
* `-p` we provide it the input path of our file
* `-o` we provide the path where to store the outputs


Let's install Bacalhau:

In [ ]:
!curl -sL https://get.bacalhau.org/install.sh | bash

Your system is linux_amd64
No BACALHAU detected. Installing fresh BACALHAU CLI...
Getting the latest BACALHAU CLI...
Installing v0.3.8 BACALHAU CLI...
Verified OK
Extracting tarball ...
NOT verifying Bin
bacalhau installed into /usr/local/bin successfully.
Client Version: v0.3.8
Server Version: v0.3.8


In [ ]:
%%bash --out job_id
bacalhau docker run \ 
--wait \
--id-only \
--gpu 1 \
--timeout 3600 \
--wait-timeout-secs 3600 \
jsacex/whisper \
-i bafybeielf6z4cd2nuey5arckect5bjmelhouvn5rhbjlvpvhp7erkrc4nu \
-- python openai-whisper.py -p inputs/Apollo_11_moonwalk_montage_720p.mp4 -o outputs

215dc3ca-e59a-4a06-9272-0be8304f1e1d


In [ ]:
%%bash
bacalhau list --id-filter ${JOB_ID} --wide

 CREATED   ID        JOB                      STATE      VERIFIED  PUBLISHED               
 08:04:36  d4ae780f  Docker jsacex/whispe...  Completed            /ipfs/QmabtzjaAj94sG... 



Where it says `Completed `, that means the job is done, and we can get the results.

To find out more information about your job, run the following command:

In [ ]:
%%bash
bacalhau describe ${JOB_ID}

APIVersion: V1alpha1
ClientID: 65f7e03a4abefc46b3ebcccfc84877fb15e9912fe541146996ce0b8279e51847
CreatedAt: "2022-10-28T08:06:30.723682632Z"
Deal:
  Concurrency: 1
ExecutionPlan:
  ShardsTotal: 1
ID: 215dc3ca-e59a-4a06-9272-0be8304f1e1d
JobState:
  Nodes:
    QmSyJ8VUd4YSPwZFJSJsHmmmmg7sd4BAc2yHY73nisJo86:
      Shards:
        "0":
          NodeId: QmSyJ8VUd4YSPwZFJSJsHmmmmg7sd4BAc2yHY73nisJo86
          PublishedResults: {}
          State: Cancelled
          VerificationResult: {}
    QmUDAXvv31WPZ8U9CzuRTMn9iFGiopGE7rHiah1X8a6PkT:
      Shards:
        "0":
          NodeId: QmUDAXvv31WPZ8U9CzuRTMn9iFGiopGE7rHiah1X8a6PkT
          PublishedResults: {}
          State: Cancelled
          VerificationResult: {}
    QmVAb7r2pKWCuyLpYWoZr9syhhFnTWeFaByHdb8PkkhLQG:
      Shards:
        "0":
          NodeId: QmVAb7r2pKWCuyLpYWoZr9syhhFnTWeFaByHdb8PkkhLQG
          PublishedResults:
            CID: bafybeievylhpwyuwegmbbozhsnf5rgeul4uyxmretjp5dybnze5h23opzu
            Name: job-215d

To download the results of your job, run the following command:

In [ ]:
%%bash
rm -rf results && mkdir -p results
bacalhau get $JOB_ID --output-dir results

Fetching results of job '215dc3ca-e59a-4a06-9272-0be8304f1e1d'...2022/10/28 08:13:02 failed to sufficiently increase receive buffer size (was: 208 kiB, wanted: 2048 kiB, got: 416 kiB). See https://github.com/lucas-clemente/quic-go/wiki/UDP-Receive-Buffer-Size for details.


After the download has finished we can see the following contents in results directory:

In [ ]:
%%bash
ls results/

job-215dc3ca-e59a-4a06-9272-0be8304f1e1d-shard-0-host-QmVAb7r2pKWCuyLpYWoZr9syhhFnTWeFaByHdb8PkkhLQG
shards
stderr
stdout
volumes


We can view the outputs:

In [ ]:
%%bash
cat results/combined_results/outputs/Apollo_11_moonwalk_montage_720p.vtt

WEBVTT

00:00.000 --> 00:07.000
As the foot of the ladder, the lamb foot beds are only depressed in the surface about one

00:14.760 --> 00:21.760
or two inches, although the surface appears to be very, very fine grained as you get close

00:23.360 --> 00:28.360
to it. It's almost like a powder. The ground mass is very fine.

00:28.360 --> 00:35.360
Okay, I'm going to leave that one foot up there and both hands down about the fourth

00:47.760 --> 00:48.760
rung up.

00:48.760 --> 00:49.760
There you go.

00:49.760 --> 00:52.760
Okay, now I think I'll do the same.

00:52.760 --> 00:59.760
For those who haven't read the plaque, we'll read the plaque that's on the front landing

01:06.840 --> 01:13.840
gear of this lamb. There's two hemispheres, one showing each of the two hemispheres of

01:13.840 --> 01:20.840
Earth. Underneath it says, Airman from the planet Earth, first set foot upon the moon

01:25.280 --> 01:28.280
July 1969, D.C. We came in deep.

01:28.280 --> 01:35.280
I guess y